<a href="https://colab.research.google.com/github/freakezoide/codigos-de-coolab/blob/main/limpieza_de_datos_primaria_desde_2019_hasta_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
"""Limpieza avanzada de datos para análisis de interacción"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Carga de datos con controles mejorados
archivos = ["2019primaria.csv", "2020Primaria.csv", "2021 primaria.csv",
            "2022 primaria.csv", "2023 primaria.csv", "2024 primaria.csv"]

# Cargar con verificación de archivos
dfs = []
for archivo in archivos:
    try:
        df_temp = pd.read_csv(archivo)
        # Verificar columnas mínimas requeridas
        required_cols = ['UsuariosCREA', 'UsuariosMAT', 'UsuariosBiblioteca', 'Departamento', 'Mes', 'Año']
        if all(col in df_temp.columns for col in required_cols):
            dfs.append(df_temp)
        else:
            print(f"Advertencia: El archivo {archivo} no tiene todas las columnas requeridas")
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {archivo}")
    except Exception as e:
        print(f"Error al procesar {archivo}: {str(e)}")

if not dfs:
    raise ValueError("No se pudieron cargar datos válidos de ningún archivo")

df = pd.concat(dfs, ignore_index=True)

## 2. Limpieza avanzada de datos
def limpieza_avanzada(df):
    # Eliminar duplicados exactos y casi-duplicados
    df = df.drop_duplicates()

    # Eliminar filas donde todas las métricas de interacción son cero o faltantes
    metricas = ['UsuariosCREA', 'UsuariosMAT', 'UsuariosBiblioteca']
    df = df.dropna(subset=metricas, how='all')
    df = df[~(df[metricas] == 0).all(axis=1)]

    # Limpieza de campos categóricos
    if 'Departamento' in df.columns:
        df['Departamento'] = df['Departamento'].str.strip().str.title()
        df['Departamento'] = df['Departamento'].replace({
            'Montevideo': 'Montevideo',
            'Canelones': 'Canelones',
            'Maldonado': 'Maldonado'
            # Agregar más estandarizaciones según necesidad
        })

    # Estandarización de meses
    meses_map = {
        'Ene': 'Enero', 'Feb': 'Febrero', 'Mar': 'Marzo', 'Abr': 'Abril',
        'May': 'Mayo', 'Jun': 'Junio', 'Jul': 'Julio', 'Ago': 'Agosto',
        'Set': 'Septiembre', 'Oct': 'Octubre', 'Nov': 'Noviembre', 'Dic': 'Diciembre'
    }
    df['Mes'] = df['Mes'].str.strip().str.title().replace(meses_map)

    # Validación de rangos numéricos
    for col in metricas:
        df = df[(df[col] >= 0) & (df[col] <= 100)]  # Asumiendo que son porcentajes

    # Eliminar outliers usando IQR
    for col in metricas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        df = df[~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))]

    return df

df = limpieza_avanzada(df)

## 3. Clasificación mejorada de interacciones
def clasificar_interaccion_mejorada(serie):
    # Usar percentiles robustos (menos sensibles a outliers)
    bajo = serie.quantile(0.33)
    medio = serie.quantile(0.66)

    # Asegurar que los límites sean significativos
    if medio - bajo < 5:  # Si la diferencia es muy pequeña
        medio = bajo + 10  # Ajuste empírico

    return pd.cut(serie,
                bins=[-1, bajo, medio, float('inf')],
                labels=['Baja', 'Media', 'Alta'],
                include_lowest=True)

# Aplicar clasificación mejorada
df['InteraccionCREA'] = clasificar_interaccion_mejorada(df['UsuariosCREA'])
df['InteraccionMAT'] = clasificar_interaccion_mejorada(df['UsuariosMAT'])
df['InteraccionBiblioteca'] = clasificar_interaccion_mejorada(df['UsuariosBiblioteca'])

## 4. Análisis de calidad de datos
def reporte_calidad(df):
    print("\n=== REPORTE DE CALIDAD DE DATOS ===")
    print(f"Registros totales: {len(df)}")
    print("\nValores faltantes por columna:")
    print(df.isnull().sum())
    print("\nDistribución de interacciones:")
    for plataforma in ['CREA', 'MAT', 'Biblioteca']:
        print(f"\n{plataforma}:")
        print(df[f'Interaccion{plataforma}'].value_counts(normalize=True) * 100)

    # Verificar consistencia temporal
    if 'Año' in df.columns and 'Mes' in df.columns:
        print("\nConsistencia temporal:")
        print(df.groupby('Año')['Mes'].value_counts().unstack())

reporte_calidad(df)

## 5. Exportar datos limpios
df.to_csv('datos_educativos_limpios.csv', index=False)
print("\nDatos limpios exportados a 'datos_educativos_limpios.csv'")

Error: No se encontró el archivo 2019primaria.csv

=== REPORTE DE CALIDAD DE DATOS ===
Registros totales: 12353

Valores faltantes por columna:
Año                        0
Mes                        0
Departamento               0
Ciclo                      0
Zona                       0
Contexto                 710
UsuariosCREA               0
UsuariosMAT                0
UsuariosBiblioteca         0
InteraccionCREA            0
InteraccionMAT             0
InteraccionBiblioteca      0
dtype: int64

Distribución de interacciones:

CREA:
InteraccionCREA
Baja     36.428398
Alta     33.352222
Media    30.219380
Name: proportion, dtype: float64

MAT:
InteraccionMAT
Media    56.957824
Baja     35.562212
Alta      7.479964
Name: proportion, dtype: float64

Biblioteca:
InteraccionBiblioteca
Baja     77.333441
Media    22.666559
Alta      0.000000
Name: proportion, dtype: float64

Consistencia temporal:
Mes   Abril  Agosto  Diciembre  Julio  Junio  Marzo   Mayo  Noviembre  \
Año              